# Library

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

## User Defined Function

In [2]:
def save_graph_image(graph, filename="lg_graph.png"):
    """
    Generate a Mermaid diagram of the graph and save it as a PNG image.
    Uses the graph's built-in Mermaid export functionality.
    """
    try:
        # Get the Mermaid PNG representation of the graph
        # This requires the 'grandalf' package for rendering
        png_data = graph.get_graph(xray=True).draw_mermaid_png()

        # Write the PNG data to file
        with open(filename, "wb") as f:
            f.write(png_data)

        print(f"Graph image saved to {filename}")
    except Exception as e:
        print(f"Could not save graph image: {e}")
        print("You may need to install additional dependencies: pip install grandalf")

In [3]:
def get_device():
    """
    Detect and return the best available compute device.
    Returns 'cuda' for NVIDIA GPUs, 'mps' for Apple Silicon, or 'cpu' as fallback.
    """
    if torch.cuda.is_available():
        print("Using CUDA (NVIDIA GPU) for inference")
        return "cuda"
    elif torch.backends.mps.is_available():
        print("Using MPS (Apple Silicon) for inference")
        return "mps"
    else:
        print("Using CPU for inference")
        return "cpu"

In [4]:
# =============================================================================
# STATE DEFINITION
# =============================================================================
# The state is a TypedDict that flows through all nodes in the graph.
# Each node can read from and write to specific fields in the state.
# LangGraph automatically merges the returned dict from each node into the state.

class AgentState(TypedDict):
    """
    State object that flows through the LangGraph nodes.

    Fields:
    - user_input: The text entered by the user (set by get_user_input node)
    - should_exit: Boolean flag indicating if user wants to quit (set by get_user_input node)
    - llm_response: The response generated by the LLM (set by call_llm node)

    State Flow:
    1. Initial state: all fields empty/default
    2. After get_user_input: user_input and should_exit are populated
    3. After call_llm: llm_response is populated
    4. After print_response: state unchanged (node only reads, doesn't write)

    The graph loops continuously:
        get_user_input -> [conditional] -> call_llm -> print_response -> get_user_input
                              |
                              +-> END (if user wants to quit)
    """
    user_input: str
    should_exit: bool
    verbose: bool
    is_empty: bool

    use_qwen: bool
    llm_response: str


In [5]:
def create_llm():
    """
    Create and configure the LLM using HuggingFace's transformers library.
    Downloads llama-3.2-1B-Instruct from HuggingFace Hub and wraps it
    for use with LangChain via HuggingFacePipeline.
    """
    # Get the optimal device for inference
    device = get_device()

    # Model identifier on HuggingFace Hub
    model_id = "meta-llama/Llama-3.2-1B-Instruct"

    print(f"Loading model: {model_id}")
    print("This may take a moment on first run as the model is downloaded...")

    # Load the tokenizer - converts text to tokens the model understands
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Load the model itself with appropriate settings for the device
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        dtype=torch.float16 if device != "cpu" else torch.float32,
        device_map=device if device == "cuda" else None,
    )

    # Move model to MPS device if using Apple Silicon
    if device == "mps":
        model = model.to(device)

    # Create a text generation pipeline that combines model and tokenizer
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=32,  # Maximum tokens to generate in response
        do_sample=True,      # Enable sampling for varied responses
        temperature=0.7,     # Controls randomness (lower = more deterministic)
        top_p=0.95,          # Nucleus sampling parameter
        pad_token_id=tokenizer.eos_token_id,  # Suppress pad_token_id warning
    )

    # Wrap the HuggingFace pipeline for use with LangChain
    llm = HuggingFacePipeline(pipeline=pipe)

    print("Model loaded successfully!")
    return llm

## User Input Function

In [6]:
# =========================
# NODE 1: get_user_input
# =========================
def get_user_input(state: AgentState) -> dict:

    if state.get("verbose", False):
        print("[TRACE] Entering get_user_input")

    print("\n" + "=" * 50)
    print("Enter your text (or 'quit' to exit):")
    print("=" * 50)

    print("\n> ", end="")
    user_input = input().strip()

    # Exit command
    if user_input.lower() in ['quit', 'exit', 'q']:
        print("Goodbye!")
        return {
            "user_input": user_input,
            "should_exit": True
        }

    # Empty input -> mark as empty and do NOT call LLM
    if user_input == "":
        return {
            "user_input": "",
            "should_exit": False,
            "is_empty": True
        }
    # Turn verbose ON
    if user_input.lower() == "verbose":
        print("Verbose mode ENABLED")
        return {
            "user_input": "",
            "should_exit": False,
            "verbose": True
        }

    # Turn verbose OFF
    if user_input.lower() == "quiet":
        print("Verbose mode DISABLED")
        return {
            "user_input": "",
            "should_exit": False,
            "verbose": False
        }

    return {
        "user_input": user_input,
        "should_exit": False,
        "is_empty": False
    }

In [7]:
# langgraph_simple_agent.py
# Program demonstrates use of LangGraph for a very simple agent.
# It writes to stdout and asks the user to enter a line of text through stdin.
# It passes the line to the LLM llama-3.2-1B-Instruct, then prints the
# what the LLM returns as text to stdout.
# The LLM should use Cuda if available, if not then if mps is available then use that,
# otherwise use cpu.
# After the LangGraph graph is created but before it executes, the program
# uses the Mermaid library to write a image of the graph to the file lg_graph.png
# The program gets the LLM llama-3.2-1B-Instruct from Hugging Face and wraps
# it for LangChain using HuggingFacePipeline.
# The code is commented in detail so a reader can understand each step.

# Import necessary libraries


# Determine the best available device for inference
# Priority: CUDA (NVIDIA GPU) > MPS (Apple Silicon) > CPU

def create_graph(llm):

    # =========================
    # NODE 2: call_llm
    # =========================
    def call_llm(state: AgentState) -> dict:

        if state.get("verbose", False):
            print("[TRACE] Entering call_llm")
            print(f"[TRACE] user_input = {state['user_input']}")

        user_input = state["user_input"]
        prompt = f"User: {user_input}\nAssistant:"

        if state.get("verbose", False):
            print("[TRACE] Prompt sent to LLM:")
            print(prompt)

        response = llm.invoke(prompt)

        if state.get("verbose", False):
            print("[TRACE] LLM returned response")

        return {"llm_response": response}

    # =========================
    # NODE 3: print_response
    # =========================
    def print_response(state: AgentState) -> dict:

        if state.get("verbose", False):
            print("[TRACE] Entering print_response")

        print("\n" + "-" * 50)
        print("LLM Response:")
        print("-" * 50)
        print(state["llm_response"])

        if state.get("verbose", False):
            print("[TRACE] Returning to get_user_input")

        return {}

    # =========================
    # ROUTING FUNCTION
    # =========================
    def route_after_input(state: AgentState) -> str:

        if state.get("verbose", False):
            print("[TRACE] Routing after input")

        if state.get("should_exit", False):
            if state.get("verbose", False):
                print("[TRACE] Routing to END")
            return END
        # 2) Empty -> self-loop back to get_user_input
        if state.get("is_empty", False):
            if state.get("verbose", False):
                 print("[TRACE] Empty input detected. Routing back to get_user_input.")
            return "get_user_input"

        if state.get("verbose", False):
            print("[TRACE] Routing to call_llm")

        return "call_llm"

    # =========================
    # GRAPH BUILDING
    # =========================
    graph_builder = StateGraph(AgentState)

    graph_builder.add_node("get_user_input", get_user_input)
    graph_builder.add_node("call_llm", call_llm)
    graph_builder.add_node("print_response", print_response)

    graph_builder.add_edge(START, "get_user_input")

    graph_builder.add_conditional_edges(
        "get_user_input",
        route_after_input,
        {
            "call_llm": "call_llm",
            "get_user_input": "get_user_input",
            END: END
        }
    )

    graph_builder.add_edge("call_llm", "print_response")
    graph_builder.add_edge("print_response", "get_user_input")

    graph = graph_builder.compile()
    return graph



In [ ]:
def main():
    """
    Main function that orchestrates the simple agent workflow:
    1. Initialize the LLM
    2. Create the LangGraph
    3. Save the graph visualization
    4. Run the graph once (it loops internally until user quits)

    The graph handles all looping internally through its edge structure:
    - get_user_input: Prompts and reads from stdin
    - call_llm: Processes input through the LLM
    - print_response: Outputs the response, then loops back to get_user_input

    The graph only terminates when the user types 'quit', 'exit', or 'q'.
    """
    print("=" * 50)
    print("LangGraph Simple Agent with Llama-3.2-1B-Instruct")
    print("=" * 50)
    print()

    # Step 1: Create and configure the LLM
    llm = create_llm()

    # Step 2: Build the LangGraph with the LLM
    print("\nCreating LangGraph...")
    graph = create_graph(llm)
    print("Graph created successfully!")

    # Step 3: Save a visual representation of the graph before execution
    # This happens BEFORE any graph execution, showing the graph structure
    print("\nSaving graph visualization...")
    save_graph_image(graph)

    # Step 4: Run the graph - it will loop internally until user quits
    # Create initial state with empty/default values
    # The graph will loop continuously, updating state as it goes:
    #   - get_user_input displays banner, populates user_input and should_exit
    #   - call_llm populates llm_response
    #   - print_response displays output, then loops back to get_user_input
    initial_state: AgentState = {
        "user_input": "",
        "should_exit": False,
        "llm_response": "",
        "is_empty": False,
        "verbose": False
    }

    # Single invocation - the graph loops internally via print_response -> get_user_input
    # The graph only exits when route_after_input returns END (user typed quit/exit/q)
    graph.invoke(initial_state)

# Entry point - only run main() if this script is executed directly
if __name__ == "__main__":
    main()